# Using data collators with 🤗 Transformers and Datasets
> Calculating top losses by example

- toc: true 
- badges: false
- comments: false
- categories: [til,nlp,huggingface,transformers,datasets]
- image: images/icml.png

In [1]:
#hide
import warnings
import datasets
import transformers

warnings.filterwarnings("ignore")
datasets.logging.set_verbosity_error()
transformers.logging.set_verbosity_error()

Recently, [Sylvain Gugger](https://twitter.com/GuggerSylvain?s=20) from HuggingFace has created some nice tutorials on using `transformers` for [text classification](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb) and [named entity recognition](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=545PP3o8IrJV). One trick that caught my attention was the use of a _data collator_ in the trainer, which automatically pads the model inputs in a batch to the length of the longest example. This bypasses the need to set a _global_ maximum sequence length, and in practice leads to faster training since we perform fewer redundant computations on the padded tokens and attention masks.

I wanted to use to use a data collator for both training _and_ error analysis (e.g. by inspecting the top losses of the model). One problem: during training, each batch is collated on the fly so how do I do pad my inputs in subsequent `Dataset.map` operations?

The solution I ended up with was to simply grab the data collator from the trainer and use in in my post-processing functions:

```python
from transformers import Trainer

trainer = Trainer(...)
trainer.train()

data_collator = trainer.data_collator

def processing_function(batch):
    # pad inputs and (possibly) labels
    batch = data_collator(batch)
    ...
    return batch
```

For an end-to-end example, let's grab 1,000 examples from the IMDB dataset:

In [56]:
from datasets import load_dataset

imdb = load_dataset('imdb', split='train').train_test_split(train_size=800, test_size=200)
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
})

Next, let's load a pretrained model and its corresponding tokenizer:

In [11]:
#hide_output
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

num_labels = 2
model_name = 'distilbert-base-cased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

Before fine-tuning the model, we need to tokenize and encode the dataset, so let's do that with a simple `Dataset.map` operation:

In [71]:
imdb_enc = imdb.map(lambda x : tokenizer(x['text'], truncation=True), batched=True)
imdb_enc

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text'],
        num_rows: 200
    })
})

The final step is to define the metrics

In [72]:
import numpy as np
from datasets import load_metric

accuracy_score = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_score.compute(predictions=predictions, references=labels)

the arguments for the trainer

In [73]:
from transformers import TrainingArguments

batch_size = 16
logging_steps = len(imdb_enc['train']) // batch_size

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
)

and the trainer itself:

> Important: The trainer will remove _in-place_ any dataset columns of `str` type, so in this example `imdb_enc` loses the `text` column.

In [74]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=imdb_enc['train'],
    eval_dataset=imdb_enc['test'],
    tokenizer=tokenizer
)

trainer.train();

Epoch,Training Loss,Validation Loss,Accuracy
1,0.390015,0.328747,0.875000


By default, the `Trainer` class uses the simple `default_data_collator` to collate batches of dict-like objects, but by passing the tokenizer we get a `DataCollatorWithPadding` instead:

In [63]:
data_collator = trainer.data_collator
type(data_collator)

transformers.data.data_collator.DataCollatorWithPadding

To see how this collator works, let's pass a dummy batch and observe that both the `input_ids` and `attention_mask` are padded as expected:

In [69]:
batch = {'input_ids':[[0,1,2], [0,1,2,3,4,5]]}
data_collator(batch)

{'input_ids': tensor([[0, 1, 2, 0, 0, 0],
        [0, 1, 2, 3, 4, 5]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1]])}

Finally, we can calculate the loss per example with the following function:{% fn 1 %}

{{ 'The non-padded version of this function comes from [Leandro von Werra](https://twitter.com/lvwerra?s=20).' | fndetail: 1 }}

In [80]:
def loss_per_example(batch):
    batch = data_collator(batch)
    input_ids = torch.tensor(batch["input_ids"], device=device)
    attention_mask = torch.tensor(batch["attention_mask"], device=device)
    labels = torch.tensor(batch["labels"], device=device)

    with torch.no_grad():
        output = model(input_ids, attention_mask)
        batch["predicted_label"] = torch.argmax(output.logits, axis=1)

    loss = torch.nn.functional.cross_entropy(output.logits, labels, reduction="none")
    batch["loss"] = loss
    
    # datasets requires list of NumPy array data types
    for k, v in batch.items():
        batch[k] = v.cpu().numpy()

    return batch

losses_ds = imdb_enc['test'].map(loss_per_example, batched=True, batch_size=batch_size)

It's then a simple matter to convert `losses_ds` to a `DataFrame` and sort by loss to find the examples where the model is most confused:

In [82]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

losses_ds.set_format('pandas')
losses_df = losses_ds[:][['label', 'predicted_label', 'loss']]
losses_df['text'] = imdb['test']['text']
losses_df.sort_values("loss", ascending=False).head()

,label,predicted_label,loss,text
147,1,0,3.477502,"Was the script more fitting for a 30 minute sitcom? Yes, but they still make it work! I thought the actors did a fantastic job with an otherwise bland script, especially Jack Black and Christopher Walken. Most people on the board seem to really hate this film. I personally can't see how that could be, but Envy is just one of those film that you either love it or hate it. Much like Napoleon Dynamite and every Leslie Neilsen movie ever made. You either think it's one of the worst movies ever made or one of the funniest. Don't avoid this movie because of the reviews. Watch it and see if you're one of the ones who really like it! If you do, I guarantee it's worth your money. If you don't like it... well, now you know."
143,1,0,2.925410,"I would just like to say, that no matter how low budget the film is, it needs to be shown throughout this world the point to these movies. We don't read that much anymore, instead people want to see movies. Having this series out on DVD, has made me want to read the whole series, and want more. PLEASE MAKE ALL 8 MOVIES. Please don't change any of the characters either, it ruins the effect. Because I have grown to love the actors who have played the characters. PLEASE MAKE ALL 8 MOVIES. I want to see the message, and watch the message that these books and now movies are here to portray. We don't get that enough anymore. AWESOME JOB!!!"
57,0,1,2.873445,"I like Brad Pitt enormously. He is an actor with brains and wit, not to mention face, pectorals and all the rest. Since I saw him in ""Thelma and Louise"" a thought has been bothering me, who does he remind me of? ""Troy"" did it for me. He is the new Brigitte Bardot. The differences are obvious of course. Male, American etc but Brigitte Bardot comes to mind nonetheless. He is so beautiful that he is at his most effective when he plays against it. ""Kalifornia"" ""12 Monkeys"" ""Fight Club"" ""Snatch"" His self deprecating humor makes him human, almost accessible. Fortunately ""Troy"" will soon be forgotten. Only still photographs with Pitt, semi naked in ravishing sprint positions will decorate the walls of legions of salivating fans. Strange, ""Das Boot"" is one of the great films of the second part of the 20th Century. What is Wolfgang Petersen doing directing this? Well, I suppose it would be very hard to say no at the chance of working with the new Brigitte Bardot."
151,1,0,2.861723,"SOLDIER is not as bad as many have made it out to be. I found the film to have some of the sacarstic, cynical humour like that in Paul Verhoven's Starship Troopers. The lack of dialogue and over the top action is deliberate and adds to the comic-book atmosphere.<br /><br />One particular trivia-bit stands out for me - Todd has the names of several space-war campaigns tattoo'd onto his chest and one of these battles is TANNHAUSER GATE. For the oblivious ones out there, Tannhauser Gate is mentioned in Roy Batty's elegiac last lines in Blade Runner. To imagine that Todd could have fought alongside android troops like Roy is mind boggling to say the least. Maybe script writer David Peoples was nostalgic?<br /><br />I'll give this one 3 out of 5."
53,0,1,2.849806,"Reed Diamond plays a man suffering from amnesia who's been in a mental asylum for over a decade after he was found wondering the back roads with blood on his hands. The doctors want to test out an experimental new drug that'll return his lost memories if it works. But when the drugs give him hallucinations of a demon, he chooses to escape instead. While outside he befriends a young boy whose stepfather (Greg Grunberg) mistreats his mother, won't let her near the darkroom in his basement & acts suspicious in general.<br /><br />While the general 'mystery' of the film is a tad easy to identify way before it's revealed, I found Mr. Diamond's acting to be enthralling enough to keep my attention throughout. (In the interest of full disclosure, I've been a huge fan of